In [1]:
from bayes_opt import BayesianOptimization

In [2]:
import itertools
import random
import warnings

from collections import namedtuple

from isolation import Board
from sample_players import RandomPlayer
from sample_players import null_score
from sample_players import open_move_score
from sample_players import improved_score
from game_agent import CustomPlayer

In [3]:
def custom_score(game, player, agent_weight=1, agent_exp=2, agent_base=0, opp_weight=2, opp_exp=1, opp_base=0):
    player_moves = game.get_legal_moves(player)
    opp_moves = game.get_legal_moves(game.get_opponent(player))
    return float(agent_weight*len(player_moves)**agent_exp + agent_base**len(player_moves) - 
                 opp_weight*len(opp_moves)**opp_exp + opp_base**len(opp_moves))

In [4]:
import random
import logging
from math import log

class Timeout(Exception):
    """Subclass base exception for code clarity."""
    pass

class CustomPlayer1:
    """Game-playing agent that chooses a move using your evaluation function
    and a depth-limited minimax algorithm with alpha-beta pruning. You must
    finish and test this player to make sure it properly uses minimax and
    alpha-beta to return a good move before the search time limit expires.

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        layers in the game tree to explore for fixed-depth search. (i.e., a
        depth of one (1) would only explore the immediate sucessors of the
        current state.)

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    iterative : boolean (optional)
        Flag indicating whether to perform fixed-depth search (False) or
        iterative deepening search (True).

    method : {'minimax', 'alphabeta'} (optional)
        The name of the search method to use in get_move().

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """
    
    
    def __init__(self, search_depth=3, score_fn=custom_score,
                 iterative=True, method='minimax', timeout=10., 
                 agent_weight=1, agent_exp=2, agent_base=0, opp_weight=2, opp_exp=1, opp_base=0):
        self.search_depth = search_depth
        self.iterative = iterative
        self.score = score_fn
        self.method = method
        self.time_left = None
        self.TIMER_THRESHOLD = timeout
        self.agent_weight = agent_weight
        self.agent_exp = agent_exp
        self.agent_base = agent_base
        self.opp_weight = opp_weight
        self.opp_exp = opp_exp
        self.opp_base = opp_base

    def get_move(self, game, legal_moves, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        This function must perform iterative deepening if self.iterative=True,
        and it must use the search method (minimax or alphabeta) corresponding
        to the self.method value.

        **********************************************************************
        NOTE: If time_left < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        -------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """

        self.time_left = time_left

        # TODO: finish this function!

        # Perform any required initializations, including selecting an initial
        # move from the game board (i.e., an opening book), or returning
        # immediately if there are no legal moves
        depth = 1
        best_move = (-1,-1)
        try:
            # The search method call (alpha beta or minimax) should happen in
            # here in order to avoid timeout. The try/except block will
            # automatically catch the exception raised by the search method
            # when the timer gets close to expiring
            if self.iterative:
                if self.method=='minimax':
                    while True:
                        best_score, best_move = self.minimax(game, depth)
                        depth+=1
                else:
                    while True:
                        best_score, best_move = self.alphabeta(game, depth)
                        depth+=1
            else:
                if self.method=='minimax':
                    best_score, best_move = self.minimax(game, self.search_depth)
                else:
                    best_score, best_move = self.alphabeta(game, self.search_depth)

        except Timeout:
            # Handle any actions required at timeout, if necessary
            return best_move

        # Return the best move from the last completed search iteration
        return best_move
    
    def minimax(self, game, depth, maximizing_player=True):
        """Implement the minimax search algorithm as described in the lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        -------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project unit tests; you cannot call any other
                evaluation function directly.
        """
        # TERMINAL-TEST
        if game.is_winner(self):
            return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base), game.get_player_location(self)
        if game.is_loser(self):
            return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base), (-1,-1)
        def max_value(game, current_depth):
            """Finds the best move for agent at current_depth.
            """
            if self.time_left() < self.TIMER_THRESHOLD:
                raise Timeout()
            if game.is_winner(self):
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            if game.is_loser(self):
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            if current_depth >= depth:  
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            score = float("-inf")
            for m in game.get_legal_moves(self):
                score = max(score, min_value(game.forecast_move(m), current_depth+1))
            return score
        def min_value(game, current_depth):
            """Finds the best move for opponent at current_depth.
            """
            if self.time_left() < self.TIMER_THRESHOLD:
                raise Timeout()
            if game.is_winner(self):
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            if game.is_loser(self):
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            if current_depth >= depth:
                return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
            score = float("inf")
            for m in game.get_legal_moves():
                score = min(score, max_value(game.forecast_move(m), current_depth+1))
            return score
        moves = game.get_legal_moves()
        imm_scores = [min_value(game.forecast_move(m),1) for m in moves]
        best_score = max(imm_scores)
        best_move = moves[imm_scores.index(best_score)]
        return best_score, best_move

    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf"), maximizing_player=True):
        """Implement minimax search with alpha-beta pruning as described in the
        lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        -------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves

        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project unit tests; you cannot call any other
                evaluation function directly.
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()
        if game.is_winner(self):
            return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base), game.get_player_location(self)
        if game.is_loser(self):
            return self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base), (-1,-1)
        low_score, high_score = float("inf"), float('-inf')
        best_move = (-1,-1)
        if depth == 1:
            if maximizing_player:
                for move in game.get_legal_moves(self):
                    score = self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
                    if score >= beta:
                        return score, move
                    if score > high_score:
                        high_score, best_move = score, move
                return high_score, best_move
            else:
                for move in game.get_legal_moves():
                    score = self.score(game, self, self.agent_weight, self.agent_exp, self.agent_base,
                              self.opp_weight, self.opp_exp, self.opp_base)
                    if score <= alpha:
                        return score, move
                    if score < low_score:
                        low_score, best_move = score, move
                return low_score, best_move
        if maximizing_player:
            for move in game.get_legal_moves(self):
                score, _ = self.alphabeta(game.forecast_move(move), depth-1, alpha, beta, maximizing_player=False)
                if score>=beta:
                    return score, move
                if score>high_score:
                    high_score, best_move = score, move
                alpha = max(high_score, alpha)
            return high_score, best_move
        else:
            for move in game.get_legal_moves():
                score, _ = self.alphabeta(game.forecast_move(move), depth-1, alpha, beta, maximizing_player=True)
                if score<=alpha:
                    return score, move
                if score<low_score:
                    low_score, best_move = score, move
                beta = min(low_score, beta)
            return low_score, best_move


In [5]:
NUM_MATCHES = 5  # number of matches against each opponent
TIME_LIMIT = 150  # number of milliseconds before timeout

TIMEOUT_WARNING = "One or more agents lost a match this round due to " + \
                  "timeout. The get_move() function must return before " + \
                  "time_left() reaches 0 ms. You will need to leave some " + \
                  "time for the function to return, and may need to " + \
                  "increase this margin to avoid timeouts during  " + \
                  "tournament play."

DESCRIPTION = """
This script evaluates the performance of the custom heuristic function by
comparing the strength of an agent using iterative deepening (ID) search with
alpha-beta pruning against the strength rating of agents using other heuristic
functions.  The `ID_Improved` agent provides a baseline by measuring the
performance of a basic agent using Iterative Deepening and the "improved"
heuristic (from lecture) on your hardware.  The `Student` agent then measures
the performance of Iterative Deepening and the custom heuristic against the
same opponents.
"""

Agent = namedtuple("Agent", ["player", "name"])


def play_match(player1, player2):
    """
    Play a "fair" set of matches between two agents by playing two games
    between the players, forcing each agent to play from randomly selected
    positions. This should control for differences in outcome resulting from
    advantage due to starting position on the board.
    """
    num_wins = {player1: 0, player2: 0}
    num_timeouts = {player1: 0, player2: 0}
    num_invalid_moves = {player1: 0, player2: 0}
    games = [Board(player1, player2), Board(player2, player1)]

    # initialize both games with a random move and response
    for _ in range(2):
        move = random.choice(games[0].get_legal_moves())
        games[0].apply_move(move)
        games[1].apply_move(move)

    # play both games and tally the results
    for game in games:
        winner, _, termination = game.play(time_limit=TIME_LIMIT)

        if player1 == winner:
            num_wins[player1] += 1

            if termination == "timeout":
                num_timeouts[player2] += 1
            else:
                num_invalid_moves[player2] += 1

        elif player2 == winner:

            num_wins[player2] += 1

            if termination == "timeout":
                num_timeouts[player1] += 1
            else:
                num_invalid_moves[player1] += 1

    if sum(num_timeouts.values()) != 0:
        warnings.warn(TIMEOUT_WARNING)

    return num_wins[player1], num_wins[player2]


def play_round(agents, num_matches):
    """
    Play one round (i.e., a single match between each pair of opponents)
    """
    agent_1 = agents[-1]
    wins = 0.
    total = 0.

    print("\nPlaying Matches:")
    print("----------")

    for idx, agent_2 in enumerate(agents[:-1]):

        counts = {agent_1.player: 0., agent_2.player: 0.}
        names = [agent_1.name, agent_2.name]
        print("  Match {}: {!s:^11} vs {!s:^11}".format(idx + 1, *names), end=' ')

        # Each player takes a turn going first
        for p1, p2 in itertools.permutations((agent_1.player, agent_2.player)):
            for _ in range(num_matches):
                score_1, score_2 = play_match(p1, p2)
                counts[p1] += score_1
                counts[p2] += score_2
                total += score_1 + score_2

        wins += counts[agent_1.player]

        print("\tResult: {} to {}".format(int(counts[agent_1.player]),
                                          int(counts[agent_2.player])))

    return 100. * wins / total

def main(agent_weight=1, agent_exp=2, agent_base=0, opp_weight=2, opp_exp=1, opp_base=0):

    HEURISTICS = [("Null", null_score),
                  ("Open", open_move_score),
                  ("Improved", improved_score)]
    AB_ARGS = {"search_depth": 5, "method": 'alphabeta', "iterative": False}
    MM_ARGS = {"search_depth": 3, "method": 'minimax', "iterative": False}
    CUSTOM_ARGS = {"method": 'alphabeta', 'iterative': True}
    CUSTOM_ARGS1 = {"method": 'alphabeta', 'iterative': True, 
                   'agent_weight':agent_weight, 'agent_exp':agent_exp, 'agent_base':agent_base, 
                   'opp_weight':opp_weight, 'opp_exp':opp_exp, 'opp_base':opp_base}

    # Create a collection of CPU agents using fixed-depth minimax or alpha beta
    # search, or random selection.  The agent names encode the search method
    # (MM=minimax, AB=alpha-beta) and the heuristic function (Null=null_score,
    # Open=open_move_score, Improved=improved_score). For example, MM_Open is
    # an agent using minimax search with the open moves heuristic.
    mm_agents = [Agent(CustomPlayer(score_fn=h, **MM_ARGS),
                       "MM_" + name) for name, h in HEURISTICS]
    ab_agents = [Agent(CustomPlayer(score_fn=h, **AB_ARGS),
                       "AB_" + name) for name, h in HEURISTICS]
    random_agents = [Agent(RandomPlayer(), "Random")]

    # ID_Improved agent is used for comparison to the performance of the
    # submitted agent for calibration on the performance across different
    # systems; i.e., the performance of the student agent is considered
    # relative to the performance of the ID_Improved agent to account for
    # faster or slower computers.
    test_agents = [Agent(CustomPlayer(score_fn=improved_score, **CUSTOM_ARGS), "ID_Improved"),
                   Agent(CustomPlayer1(score_fn=custom_score, **CUSTOM_ARGS), "Student")]
    print(CUSTOM_ARGS1)

    print(DESCRIPTION)
    win_ratios = []
    for agentUT in test_agents:
        print("")
        print("*************************")
        print("{:^25}".format("Evaluating: " + agentUT.name))
        print("*************************")

        agents = random_agents + mm_agents + ab_agents + [agentUT]
        win_ratio = play_round(agents, NUM_MATCHES)
        win_ratios.append(win_ratio)
        print("\n\nResults:")
        print("----------")
        print("{!s:<15}{:>10.2f}%".format(agentUT.name, win_ratio))
    return win_ratios[1]/win_ratios[0]

In [ ]:
params={'agent_weight':(0,100), 'agent_exp':(0,10), 'agent_base':(0,10),
        'opp_weight':(0,100), 'opp_exp':(0,10), 'opp_base':(0,10)}
BO = BayesianOptimization(main, params)
BO.explore({'agent_weight':[1,2,0,0], 'agent_exp':[2,1,1,1], 'agent_base':[0,0,3,2],
           'opp_weight':[2,1,0,0], 'opp_exp':[1,2,1,1], 'opp_base':[0,0,2,.5]})
BO.maximize(init_points=3, n_iter=100, acq='ucb', kappa=2.576*2)

Initialization
-------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   agent_base |   agent_exp |   agent_weight |   opp_base |   opp_exp |   opp_weight | 
{'method': 'alphabeta', 'iterative': True, 'agent_weight': 1, 'agent_exp': 2, 'agent_base': 0, 'opp_weight': 2, 'opp_exp': 1, 'opp_base': 0}

This script evaluates the performance of the custom heuristic function by
comparing the strength of an agent using iterative deepening (ID) search with
alpha-beta pruning against the strength rating of agents using other heuristic
functions.  The `ID_Improved` agent provides a baseline by measuring the
performance of a basic agent using Iterative Deepening and the "improved"
heuristic (from lecture) on your hardware.  The `Student` agent then measures
the performance of Iterative Deepening and the custom heuristic against the
same opponents.


*************************
 Evaluating: ID_Improved 
********

In [35]:
BO.res['max'] # First run, acq='ei'

{'max_params': {'agent_base': 7.4921842058035448,
  'agent_exp': 7.4205991775955464,
  'agent_weight': 82.874576048706842,
  'opp_base': 5.7974792415172294,
  'opp_exp': 7.6446303872568819,
  'opp_weight': 73.420012687089994},
 'max_val': 1.1363636363636365}

In [41]:
BO.res['max'] # Second run, including acq='ucb',
# BO.explore({'agent_weight':[1,2,0,0], 'agent_exp':[2,1,1,1], 'agent_base':[0,0,3,2],
#           'opp_weight':[2,1,0,0], 'opp_exp':[1,2,1,1], 'opp_base':[0,0,2,.5]}),
# init_points=3, n_iter=100

/Users/jasonmancuso/anaconda/lib/python3.6/site-packages/bayes_opt/bayesian_optimization.py:258: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 60 but corresponding boolean dimension is 61
  self.gp.fit(self.X[ur], self.Y[ur])


IndexError: index 60 is out of bounds for axis 1 with size 60